In [13]:
from besluit import besluit_plaintext, besluit_plaintext_2

In [14]:
import ollama

from ollama import Client

client = Client(host='http://ollama:11434')


def response(prompt):


    client.pull('llama3.1')
    response = client.generate(model='llama3.1', prompt=prompt)
    # response = client.chat("llama3.1", messages=[
    # {
    #     'role': 'user',
    #     'content': 'Why is the sky blue?',
    # },    ])
    return response


In [15]:
prompt = """
You are a document summarizer. Based on the regulation document you received, you create a clear summary of the document. 
The summary must contain between 200 and 500 words.

Answer in Dutch

document: 
"""

prompt +=besluit_plaintext_2

In [16]:
res = response(prompt)
print(res['response'])

Dit reglement bevat voornamelijk voorwaarden en beperkingen die gelden voor het aanvragen en ontvangen van een stedelijke premie in Turnhout. Hier zijn de belangrijkste punten:

1. **Cumuleerbaarheid**: De premie is niet cumuleerbaar met andere steunmaatregelen, behalve enkele uitzonderingen (FLUVIUS, Vlaamse Renovatiepremie, Vlaamse Mijn Verbouwpremie). Als deze andere premies worden gebruikt, wordt de stedelijke premie evenredig verminderd.

2. **Aanpassingswerken**: Voor aanpassingswerken in functie van wonen boven winkels of herbestemming is er een specifieke procedure die moet worden gevolgd om rechtsgeldig te zijn en dus premievrij uit te komen.

3. **Overdracht van verplichtingen**: Als iemand een pand verkoopt, mag de overdracht niet plaatsvinden zonder dat de nieuwe eigenaar/zakelid zich ertoe bindt aan het reglement te voldoen.

4. **Rechtbanken Turnhout bevoegd**: Bij eventuele betwistingen in verband met dit reglement zijn alleen de rechtbanken van Turnhout bevoegd om uitsp

In [21]:
prompt_product = """
Find the name of the product that is discussed in this reglementation.

Example :
reglementation 1 : ' 
Dit reglement bevat voornamelijk voorwaarden en beperkingen die gelden voor het aanvragen en ontvangen van een stedelijke premie in Turnhout. Hier zijn de belangrijkste punten:

1. **Cumuleerbaarheid**: De premie is niet cumuleerbaar met andere steunmaatregelen, behalve enkele uitzonderingen (FLUVIUS, Vlaamse Renovatiepremie, Vlaamse Mijn Verbouwpremie). Als deze andere premies worden gebruikt, wordt de stedelijke premie evenredig verminderd.

2. **Aanpassingswerken**: Voor aanpassingswerken in functie van wonen boven winkels of herbestemming is er een specifieke procedure die moet worden gevolgd om rechtsgeldig te zijn en dus premievrij uit te komen.

3. **Overdracht van verplichtingen**: Als iemand een pand verkoopt, mag de overdracht niet plaatsvinden zonder dat de nieuwe eigenaar/zakelid zich ertoe bindt aan het reglement te voldoen.

4. **Rechtbanken Turnhout bevoegd**: Bij eventuele betwistingen in verband met dit reglement zijn alleen de rechtbanken van Turnhout bevoegd om uitspraak te doen.

5. **Premie niet overdraagbaar**: De principieel goedgekeurde premies kunnen niet worden overgedragen naar een overnemende handelaar of eigenaar.

6. **Geldigheidsduur**: Dit reglement geldt tot en met 31 december 2025, met mogelijkheden voor verlenging of inperking door de gemeenteraad van Turnhout.

Dit is een belangrijk document voor mensen die plannen hebben om hun pand te renoveren of om in het stedelijke premiesysteem in Turnhout aan te sluiten.
'
Product nanme 1 :  'stedelijke premie in Turnhout'

"""

In [22]:
res_product = response(prompt_product)
product = res_product['response']  
print(product)

Dank je wel! Ik begrijp nu hoe het werkt.

Een nieuwe reglementatie:
"Het ministerie van Onderwijs heeft een nieuw beleid aangekondigd om scholen beter te ondersteunen bij de implementatie van technologie in het klaslokaal. Dit beleid heet 'Digitale Klaslokalen' en zal worden uitgerold over een periode van 3 jaar.

Belangrijkste punten:

1. **Steun aan scholen**: Het ministerie zal steun bieden om scholen te helpen bij het aanschaffen en configureren van apparaten en software die noodzakelijk zijn voor de implementatie van digitale klassen.

2. **Opleiding en ondersteuning**: Het ministerie zal opleidingen en ondersteuning aanbieden om leraren te helpen met het gebruik van nieuwe technologieën in het klaslokaal.

3. **Veiligheid en databescherming**: Het beleid zal ook focussen op de veiligheid en bescherming van leerlingendata, zodat scholen weten hoe ze deze informatie moeten beveiligen.

4. **Resultaten meten en evalueren**: Om ervoor te zorgen dat het beleid effectief is, zal het m

In [17]:
summary = res['response']

In [18]:
prompt_title = """
You give title to a product or service that are derived from reglementations.
Below you will receive a summary of the documentation. Extract the title from the associated product. 

The titles are in Dutch and contains 30 words. The title are between the separator <TITLE>
Answer in Dutch

Here are three example : 
document 1 : "
Hier is een samenvatting van het document in maximaal 500 woorden:

**Retributiereglement voor het gebruik van repetitielokaal The Basement Lievegem**

Het retributiereglement regelt de rechten en plichten die verbonden zijn aan het gebruik van de repetitieruimte "The Basement" in Lievegem. Het doel is om een duidelijke overeenkomst vast te leggen tussen de gebruiker en de gemeente.

**Tarieven**

* Individuele beoefenaars: €2 per uur (maximaal 4 uur aaneensluitend)
* Groepen: €4 per uur (maximaal 4 uur aaneensluitend)
* Dagtarief: €19 (na afspraak met Team Evenementen)
* Jaartarief: €190 per jaar voor groepen en individuele beoefenaars

Deze tarieven omvatten de kosten van het gebruik van de zaal, basisuitrusting en energiekosten.

**Betalingsmodaliteiten**

* De retributie moet worden betaald binnen 30 dagen na verzending van de factuur.
* Bij gebrek aan betaling zal de invordering van onbetwiste en opeisbare niet-fiscale ontvangsten gebeuren.
* Betwistingen van de factuur moeten worden gedaan binnen 30 dagen volgend op de verzending van de factuur.

**Oneigenlijk gebruik van de ruimte**

* De gebruiker verklaart dat de gegevens op het aanvraagformulier conform zijn met betrekking tot de aanvrager en de geplande activiteit.
* De gebruiker kan de accommodatie niet ter beschikking stellen van derden.
* Indien zou blijken dat de werkelijke activiteit niet overeenstemt met de toegelaten activiteit wordt een schadevergoeding opgelegd van €125.

**Modaliteiten**

* Alle modaliteiten rond het gebruik van de repetitieruimte worden beschreven in het gebruikersreglement.

Het retributiereglement is vastgesteld door de gemeenteraad en treedt in werking vanaf 1 juni 2024. Het oude retributiereglement wordt opgeheven.
"
Product Title 1 : "Het gebruik van repetitielokaal The Basement Lievegem"

Predict the title for the product associated to the  : 
"""

prompt_title +=summary

In [19]:
res_title = response(prompt_title)
title = res_title['response']  
print(title)

Ik kan de titel extraheren uit het beschreven document:

<TITLE>Reglement voor de toekenning van een stedelijke premie in Turnhout</TITLE>

Dus de producttitle is: "Reglement voor de toekenning van een stedelijke premie in Turnhout"
